In [1]:
from kubernetes.client import V1ObjectMeta
import kubeflow.katib as kc
from kubeflow.katib import constants
from kubeflow.katib import utils
from kubeflow.katib import V1alpha3AlgorithmSetting
from kubeflow.katib import V1alpha3AlgorithmSetting
from kubeflow.katib import V1alpha3AlgorithmSpec
from kubeflow.katib import V1alpha3CollectorSpec
from kubeflow.katib import V1alpha3EarlyStoppingSetting
from kubeflow.katib import V1alpha3EarlyStoppingSpec
from kubeflow.katib import V1alpha3Experiment
from kubeflow.katib import V1alpha3ExperimentCondition
from kubeflow.katib import V1alpha3ExperimentList
from kubeflow.katib import V1alpha3ExperimentSpec
from kubeflow.katib import V1alpha3ExperimentStatus
from kubeflow.katib import V1alpha3FeasibleSpace
from kubeflow.katib import V1alpha3FileSystemPath
from kubeflow.katib import V1alpha3FilterSpec
from kubeflow.katib import V1alpha3GoTemplate
from kubeflow.katib import V1alpha3GraphConfig
from kubeflow.katib import V1alpha3Metric
from kubeflow.katib import V1alpha3MetricsCollectorSpec
from kubeflow.katib import V1alpha3NasConfig
from kubeflow.katib import V1alpha3ObjectiveSpec
from kubeflow.katib import V1alpha3Observation
from kubeflow.katib import V1alpha3Operation
from kubeflow.katib import V1alpha3OptimalTrial
from kubeflow.katib import V1alpha3ParameterAssignment
from kubeflow.katib import V1alpha3ParameterSpec
from kubeflow.katib import V1alpha3SourceSpec
from kubeflow.katib import V1alpha3Suggestion
from kubeflow.katib import V1alpha3SuggestionCondition
from kubeflow.katib import V1alpha3SuggestionList
from kubeflow.katib import V1alpha3SuggestionSpec
from kubeflow.katib import V1alpha3SuggestionStatus
from kubeflow.katib import V1alpha3TemplateSpec
from kubeflow.katib import V1alpha3Trial
from kubeflow.katib import V1alpha3TrialAssignment
from kubeflow.katib import V1alpha3TrialCondition
from kubeflow.katib import V1alpha3TrialList
from kubeflow.katib import V1alpha3TrialSpec
from kubeflow.katib import V1alpha3TrialStatus
from kubeflow.katib import V1alpha3TrialTemplate

In [2]:
algorithmsettings = V1alpha3AlgorithmSetting(
    name= None,
    value = None
)

algorithm = V1alpha3AlgorithmSpec(
    algorithm_name = "tpe",
    algorithm_settings = [algorithmsettings]
)

# Metric Collector
collector = V1alpha3CollectorSpec(kind = "StdOut")
metrics_collector_spec = V1alpha3MetricsCollectorSpec(
    collector = collector
)

# Objective
objective = V1alpha3ObjectiveSpec(
    goal = 1.8,
    objective_metric_name = "loss",
    type = "minimize"
)

# Parameters
feasible_space = V1alpha3FeasibleSpace(min = "1", max = "3")
epochs_parameter_spec = V1alpha3ParameterSpec(
    feasible_space = feasible_space, 
    name = "--epochs",
    parameter_type = "int"
)

feasible_space = V1alpha3FeasibleSpace(min = "0.0001", max = "0.01")
lr_parameter_spec = V1alpha3ParameterSpec(
    feasible_space = feasible_space, 
    name = "--learning_rate",
    parameter_type = "double"
)

parameters = [epochs_parameter_spec, lr_parameter_spec]

# Trialtemplate
go_template = V1alpha3GoTemplate(
    raw_template =   "apiVersion: \"kubeflow.org/v1\"\nkind: TFJob\nmetadata:\n  name: {{.Trial}}\n  namespace: {{.NameSpace}}\nspec:\n tfReplicaSpecs:\n  Worker:\n    replicas: 1\n    restartPolicy: OnFailure\n    template:\n      spec:\n        containers:\n          - name: tensorflow\n            image: gcr.io/gsoc-kf-example/distributed_tf_2_neural_machine_translation:1.0\n            imagePullPolicy: Always\n            command:\n              - \"python3\"\n              - \"/app/text_classification_rnn.py\"\n              {{- with .HyperParameters}}\n              {{- range .}}\n              - \"{{.Name}}={{.Value}}\"\n              {{- end}}\n              {{- end}}"
)


trial_template= V1alpha3TrialTemplate(go_template=go_template)


# Experiment
experiment = V1alpha3Experiment(
    api_version="kubeflow.org/v1alpha3",
    kind="Experiment",
    metadata=V1ObjectMeta(name="neural-machine-translation",namespace="kubeflow-mailsforyashj"),

    spec=V1alpha3ExperimentSpec(
         algorithm = algorithm,
         max_failed_trial_count=3,
         max_trial_count=12,
         metrics_collector_spec= metrics_collector_spec ,
         objective = objective,
         parallel_trial_count=4,
         parameters = parameters ,
         trial_template = trial_template
    )
)

In [3]:
namespace = kc.utils.get_default_target_namespace()
kclient = kc.KatibClient()

### Create Experiment

In [4]:
kclient.create_experiment(experiment,namespace=namespace)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2020-07-26T17:22:44Z',
  'generation': 1,
  'name': 'neural-machine-translation',
  'namespace': 'kubeflow-mailsforyashj',
  'resourceVersion': '44990642',
  'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/kubeflow-mailsforyashj/experiments/neural-machine-translation',
  'uid': '9e02934b-cf64-11ea-89e4-42010a8c0018'},
 'spec': {'algorithm': {'algorithmName': 'tpe', 'algorithmSettings': [{}]},
  'maxFailedTrialCount': 3,
  'maxTrialCount': 12,
  'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
  'objective': {'goal': 1.8,
   'objectiveMetricName': 'loss',
   'type': 'minimize'},
  'parallelTrialCount': 4,
  'parameters': [{'feasibleSpace': {'max': '3', 'min': '1'},
    'name': '--epochs',
    'parameterType': 'int'},
   {'feasibleSpace': {'max': '0.01', 'min': '0.0001'},
    'name': '--learning_rate',
    'parameterType': 'double'}],
  'trialTemplate': {'goTemplate': {'rawTem

### Get Single Experiment

In [5]:
kclient.get_experiment(name="neural-machine-translation", namespace=namespace)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2020-07-26T17:22:44Z',
  'finalizers': ['update-prometheus-metrics'],
  'generation': 1,
  'name': 'neural-machine-translation',
  'namespace': 'kubeflow-mailsforyashj',
  'resourceVersion': '44990644',
  'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/kubeflow-mailsforyashj/experiments/neural-machine-translation',
  'uid': '9e02934b-cf64-11ea-89e4-42010a8c0018'},
 'spec': {'algorithm': {'algorithmName': 'tpe', 'algorithmSettings': [{}]},
  'maxFailedTrialCount': 3,
  'maxTrialCount': 12,
  'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
  'objective': {'goal': 1.8,
   'objectiveMetricName': 'loss',
   'type': 'minimize'},
  'parallelTrialCount': 4,
  'parameters': [{'feasibleSpace': {'max': '3', 'min': '1'},
    'name': '--epochs',
    'parameterType': 'int'},
   {'feasibleSpace': {'max': '0.01', 'min': '0.0001'},
    'name': '--learning_rate',
    'parameterType': 'double

### Get All Experiments 

In [6]:
kclient.get_experiment(namespace=namespace)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'items': [{'apiVersion': 'kubeflow.org/v1alpha3',
   'kind': 'Experiment',
   'metadata': {'creationTimestamp': '2020-07-26T17:22:44Z',
    'finalizers': ['update-prometheus-metrics'],
    'generation': 1,
    'name': 'neural-machine-translation',
    'namespace': 'kubeflow-mailsforyashj',
    'resourceVersion': '44990644',
    'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/kubeflow-mailsforyashj/experiments/neural-machine-translation',
    'uid': '9e02934b-cf64-11ea-89e4-42010a8c0018'},
   'spec': {'algorithm': {'algorithmName': 'tpe', 'algorithmSettings': [{}]},
    'maxFailedTrialCount': 3,
    'maxTrialCount': 12,
    'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
    'objective': {'goal': 1.8,
     'objectiveMetricName': 'loss',
     'type': 'minimize'},
    'parallelTrialCount': 4,
    'parameters': [{'feasibleSpace': {'max': '3', 'min': '1'},
      'name': '--epochs',
      'parameterType': 'int'},
     {'feasibleSpace': 

### Get Experiment Status

In [7]:
kclient.get_experiment_status(name="neural-machine-translation", namespace=namespace)

'Running'

### Is Experiment Succeded

In [11]:
kclient.is_experiment_succeeded(name="neural-machine-translation", namespace=namespace)

False

### List Trials of an Experiment

In [12]:
kclient.list_trials(name="neural-machine-translation", namespace=namespace)

[{'name': 'neural-machine-translation-8pb8z7fc', 'status': 'Succeeded'},
 {'name': 'neural-machine-translation-dqb64l9j', 'status': 'Succeeded'},
 {'name': 'neural-machine-translation-flkvp4nl', 'status': 'Succeeded'},
 {'name': 'neural-machine-translation-rj46w9wn', 'status': 'Succeeded'}]

### List All Experiments

In [13]:
kclient.list_experiments(namespace=namespace)

[{'name': 'neural-machine-translation', 'status': 'Running'}]

### Get Optimal Hyperparameter

In [14]:
kclient.get_optimal_hyperparmeters(name="neural-machine-translation", namespace=namespace)

{'currentOptimalTrial': {'bestTrialName': '',
  'observation': {'metrics': None},
  'parameterAssignments': None}}

### Delete Experiment

In [15]:
kclient.delete_experiment(name="neural-machine-translation", namespace=namespace)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2020-07-26T17:22:44Z',
  'deletionGracePeriodSeconds': 0,
  'deletionTimestamp': '2020-07-29T14:32:24Z',
  'finalizers': ['update-prometheus-metrics'],
  'generation': 2,
  'name': 'neural-machine-translation',
  'namespace': 'kubeflow-mailsforyashj',
  'resourceVersion': '47721653',
  'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/kubeflow-mailsforyashj/experiments/neural-machine-translation',
  'uid': '9e02934b-cf64-11ea-89e4-42010a8c0018'},
 'spec': {'algorithm': {'algorithmName': 'tpe', 'algorithmSettings': [{}]},
  'maxFailedTrialCount': 3,
  'maxTrialCount': 12,
  'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
  'objective': {'goal': 1.8,
   'objectiveMetricName': 'loss',
   'type': 'minimize'},
  'parallelTrialCount': 4,
  'parameters': [{'feasibleSpace': {'max': '3', 'min': '1'},
    'name': '--epochs',
    'parameterType': 'int'},
   {'feasibleSpace': {'max': '0.